In [1]:
import pandas as pd 
import geopandas as gpd
import contextily as ctx
from shapely.wkt import loads
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [12]:
edge_vol = pd.read_csv('simulation_outputs/edge_vol/edge_vol_hr5_qt0_2016.csv')
edges = pd.read_csv('network_inputs/tokyo_edges.csv')
display(edge_vol.head(1))
display(edges.head(1))

,start_nid,end_nid,veh_current,vol_true,vol_tot,flow,t_avg
0,1213424,1213426,0,1,1,4.0,782.36


,u,v,highway,length,lanes,maxspeed,geometry,start_nid,end_nid,signal,type,capacity,fft
0,131938557,6266083707,tertiary,51.5,1.0,40.0,"LINESTRING (139.851108 35.0268381, 139.8510964...",0,14135,0.5,tertiary,900.0,4.635


In [14]:
scen_nm = '2016'
edges_output = edges.copy()
edges_output['total_volume'] = 0
for (hour, quarter) in [(6,2), (6,3), (7,0), (7,1), (7,2), (7,3), (8,0), (8,1)]:
    edge_vol = pd.read_csv('simulation_outputs/edge_vol/edge_vol_hr{}_qt{}_{}.csv'.format(hour, quarter, scen_nm))
    edges_output = pd.merge(edges_output[['start_nid', 'end_nid', 'total_volume', 'geometry']], edge_vol[['start_nid', 'end_nid', 'vol_true']], how='left', on=['start_nid', 'end_nid'])
    edges_output['total_volume'] += edges_output['vol_true']
    edges_output = edges_output[['start_nid', 'end_nid', 'total_volume', 'geometry']]
    # edge_vol_geom.to_csv('visualization_outputs/edge_vol_geom/edge_vol_geom_hr{}_qt{}_{}.csv'.format(hour, quarter, scen_nm), index=False)
edges_output[edges_output['total_volume']>2000].to_csv('visualization_outputs/edge_vol_geom/edge_vol200_geom_hr{}_qt{}_{}.csv'.format(hour, quarter, scen_nm), index=False)
print(edges_output[edges_output['total_volume']>2000].head())

        ### plot
    #     fig, ax = plt.subplots(1, 1, figsize=(20,20))
    #     divider = make_axes_locatable(ax)
    #     cax = divider.append_axes('right', size='5%', pad=0.1)
    #     edge_vol_geom = gpd.GeoDataFrame(edge_vol_geom, crs='epsg:4326', geometry=edge_vol_geom['geometry'].map(loads))
    #     edge_vol_geom[edge_vol_geom['volume_capacity_ratio']>0.1].to_crs(epsg=3857).plot(column='volume_capacity_ratio', lw=0.5, ax=ax, cax=cax, cmap='magma_r', legend=True, vmin=0, vmax=1)
    #     ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite, alpha=0.2)
    #     fig.patch.set_facecolor('white')
    #     fig.patch.set_alpha(0.7)
    #     ax.set_title('Traffic flow (veh/hr) at {:02d}:{:02d}'.format(hour, quarter*15), font={'size': 30})
    #     plt.savefig('visualization_outputs/edge_vol_geom_plot/edge_vol_geom_hr{}_qt{}_{}.png'.format(hour, quarter, scen_nm), transparent=False)
    #     break
    # break

    start_nid  end_nid  total_volume  \
20          6     3723        3009.0   
25          7     3536        2559.0   
26          8     7484        4500.0   
27          8       46        8959.0   
29          9    15597       10341.0   

                                             geometry  
20  LINESTRING (139.8341576 35.1102233, 139.833821...  
25  LINESTRING (139.8481517 35.0964721, 139.848101...  
26  LINESTRING (139.8322405 35.1640835, 139.832556...  
27  LINESTRING (139.8322405 35.1640835, 139.831932...  
29  LINESTRING (139.8307849 35.1735173, 139.830882...  
